In [1]:
from main import *


from sklearn.pipeline import Pipeline
from sktime.transformations.panel.tsfresh import TSFreshFeatureExtractor
from sktime.classification.sklearn import RotationForest


from sktime.classification.kernel_based import RocketClassifier
from sktime.pipeline import make_pipeline
from sktime.transformations.panel.rocket import MiniRocketMultivariate
from sklearn.linear_model import LogisticRegression

from sktime.transformations.panel.shapelet_transform import RandomShapeletTransform

from sktime.classification.interval_based import DrCIF
from sktime.classification.shapelet_based import ShapeletTransformClassifier
from sktime.classification.dictionary_based import TemporalDictionaryEnsemble
from sktime.classification.deep_learning import InceptionTimeClassifier

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

In [2]:
machines = ["M01", "M02","M03"]
process_names = ["OP00","OP01","OP02","OP03","OP04","OP05","OP06","OP07","OP08","OP09","OP10","OP11","OP12","OP13","OP14"]
labels = ["good","bad"]
path_to_dataset = Path("./data/").absolute()

In [3]:
X_data = []
y_data = []

for process_name, machine, label in itertools.product(process_names, machines, labels):
    data_path = os.path.join(path_to_dataset, machine, process_name, label)
    data_list, data_label = data_loader_utils.load_tool_research_data(data_path, label=label, add_additional_label = True, verbose = False)
    X_data.extend(data_list)
    y_data.extend(data_label)

df = get_df(X_data, y_data)

## Hyperparameter Tuning

### FreshPRINCE

In [ ]:
fp = Pipeline([
    (
        "transform",
        TSFreshFeatureExtractor(
            default_fc_parameters="efficient",
            show_warnings = False),
    ),
    ("classifier", RotationForest ()),
])

param_grid_fp = {
    "classifier__n_estimators": [50, 200]
    }

cv_results_fp, gs_objects_fp = get_cv_results(fp, param_grid_fp, df)

with open("cv_results/fp.pkl", "wb") as f:
    pickle.dump(cv_results_fp, f)

### DrCIF

In [ ]:
drcif = DrCIF(time_limit_in_minutes = 10, n_jobs = -1)

param_grid_drcif = {
    "att_subsample_size": [5, 10], # 10 is default and 5 is half of the default
}

cv_results_drcif, gs_objects_drcif = get_cv_results(drcif, param_grid_drcif, df)

with open("cv_results/drcif.pkl", "wb") as f:
    pickle.dump(cv_results_drcif, f)

### STC

In [ ]:
trf = RandomShapeletTransform(time_limit_in_minutes = 10, n_jobs = -1)
clf = RotationForest(n_jobs = -1)

stc = Pipeline([("transform", trf), ("classifier", clf)])

param_grid_stc = {
    "transform__n_shapelet_samples": [1000],
    "classifier__n_estimators": [50, 200]
} 

cv_results_stc, gs_objects_stc = get_cv_results(stc, param_grid_stc, df)

with open ("cv_results/stc.pkl", "wb") as f:
    pickle.dump(cv_results_stc, f)

### TDE

In [ ]:
tde = TemporalDictionaryEnsemble(max_win_len_prop= 0.5, time_limit_in_minutes = 10, contract_max_n_parameter_samples = 100, n_jobs = 1) # To reduce memory overhead

param_grid_tde = {
    "max_ensemble_size" : [5, 25],
    "randomly_selected_params" : [5, 25]
}

cv_results_tde, gs_objects_tde = get_cv_results(tde, param_grid_tde, df)

with open("cv_results/tde.pkl", "wb") as f: 
    pickle.dump(cv_results_tde, f)

### MR

In [ ]:
trf = MiniRocketMultivariate(n_jobs = 1) 
clf = LogisticRegression(solver = "liblinear", n_jobs = 1)
        
MiniRocketLR = make_pipeline(trf, clf)

param_grid_MiniRocketLR = {
    "num_kernels" : [1000, 10000],
    'C': [0.01, 0.1, 1, 10, 100]
}

cv_results_mr, gs_objects_mr = get_cv_results(MiniRocketLR, param_grid_MiniRocketLR, df)

with open("cv_results/mr.pkl", "wb") as f:
    pickle.dump(cv_results_mr, f)

### ITC

In [ ]:
itc = InceptionTimeClassifier(n_epochs = 50, batch_size = 16, metrics = ["f1_score"])

param_grid_itc = {
    "n_filters" : [6, 32, 64],
}

cv_results_itc, gs_objects_itc = get_cv_results(itc, param_grid_itc, df, n_jobs = 1)

with open("cv_results/itc.pkl", "wb") as f: 
    pickle.dump(cv_results_itc, f)

## Results

In [4]:
with open("cv_results/fp.pkl", "rb") as f:
    cv_results_fp = pickle.load(f)

with open("cv_results/drcif.pkl", "rb") as f:
    cv_results_drcif = pickle.load(f)

with open("cv_results/stc.pkl", "rb") as f:
    cv_results_stc = pickle.load(f)

with open("cv_results/tde.pkl", "rb") as f:
    cv_results_tde = pickle.load(f)

with open("cv_results/mr.pkl", "rb") as f:
    cv_results_mr = pickle.load(f)

with open("cv_results/itc.pkl", "rb") as f:
    cv_results_itc = pickle.load(f)

In [5]:
fp_mean_ranks, best_fp = extract_mean_rank(cv_results_fp)
drcif_mean_ranks, best_drcif = extract_mean_rank(cv_results_drcif)
stc_mean_ranks, best_stc = extract_mean_rank(cv_results_stc)
tde_mean_ranks, best_tde = extract_mean_rank(cv_results_tde)
mr_mean_ranks, best_mr = extract_mean_rank(cv_results_mr)
itc_mean_ranks, best_itc = extract_mean_rank(cv_results_itc)

In [6]:
best_fp

{'classifier__n_estimators': 50}

In [7]:
fp = Pipeline([
    (
        "transform",
        TSFreshFeatureExtractor(
            default_fc_parameters="efficient",
            show_warnings = False),
    ),
    (
        "classifier", 
         RotationForest(n_estimators = 50
     )
    ),
])

In [8]:
best_drcif

{'att_subsample_size': 5}

In [9]:
drcif = DrCIF(att_subsample_size = 5, time_limit_in_minutes = 10, n_jobs = -1)

In [10]:
best_stc

{'classifier__n_estimators': 50, 'transform__n_shapelet_samples': 1000}

In [11]:
trf = RandomShapeletTransform(n_shapelet_samples = 1000, time_limit_in_minutes = 10, n_jobs = -1)
clf = RotationForest(n_estimators = 50, n_jobs = -1)

stc = Pipeline([("transform", trf), ("classifier", clf)])

In [12]:
best_tde

{'max_ensemble_size': 5, 'randomly_selected_params': 5}

In [13]:
tde = TemporalDictionaryEnsemble(max_ensemble_size = 5, 
                                 randomly_selected_params = 5, 
                                 max_win_len_prop= 0.5, # To reduce memory overhead
                                 time_limit_in_minutes = 10, 
                                 contract_max_n_parameter_samples = 100, 
                                 n_jobs = 1) 


In [14]:
best_mr

{'C': 0.01, 'num_kernels': 1000}

In [15]:
trf = MiniRocketMultivariate(num_kernels = 1000, n_jobs = 1) 
clf = LogisticRegression(C = 0.01, solver = "liblinear", n_jobs = 1)
        
mr = make_pipeline(trf, clf)

In [16]:
trf = MiniRocketMultivariate(n_jobs = 1) 
clf = LogisticRegression(n_jobs = 1)

mr_baseline = make_pipeline(trf, clf)

In [17]:
best_itc

{'n_filters': 32}

In [18]:
itc = InceptionTimeClassifier(n_filters = 32, n_epochs = 50, batch_size = 16, metrics = ["f1_score"])

In [19]:
test_fp, fp_objects = get_test_results(fp , df)

Testing machine-wise split


Feature Extraction: 100%|███████████████████████████████████████████████████████████| 4062/4062 [28:17<00:00,  2.39it/s]


Testing time-wise split


Feature Extraction: 100%|███████████████████████████████████████████████████████████| 3996/3996 [28:00<00:00,  2.38it/s]


Testing operation-wise split


Feature Extraction: 100%|███████████████████████████████████████████████████████████| 4083/4083 [27:52<00:00,  2.44it/s]


In [ ]:
test_drcif, drcif_objects = get_test_results(drcif, df)

In [21]:
test_stc, stc_objects = get_test_results(stc, df)

Testing machine-wise split
Testing time-wise split
Testing operation-wise split


In [22]:
test_tde, tde_objects = get_test_results(tde, df)

Testing machine-wise split
Testing time-wise split
Testing operation-wise split


In [103]:
test_mr, mr_objects = get_test_results(mr, df)

Testing machine-wise split
Testing time-wise split
Testing operation-wise split


In [78]:
test_mr_baseline, mr_baseline_objects = get_test_results(mr_baseline, df)

Testing machine-wise split
Testing time-wise split
Testing operation-wise split


In [25]:
test_itc, itc_objects = get_test_results(itc, df)

Testing machine-wise split


2024-12-02 18:59:52.145837: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-02 18:59:52.364003: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733162392.440657  471862 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733162392.461732  471862 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-02 18:59:52.641959: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

22/22 ━━━━━━━━━━━━━━━━━━━━ 2s 75ms/step
85/85 ━━━━━━━━━━━━━━━━━━━━ 4s 52ms/step
Testing time-wise split
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 66ms/step
84/84 ━━━━━━━━━━━━━━━━━━━━ 4s 47ms/step
Testing operation-wise split
22/22 ━━━━━━━━━━━━━━━━━━━━ 2s 74ms/step
86/86 ━━━━━━━━━━━━━━━━━━━━ 4s 43ms/step


In [26]:
results = pd.DataFrame(
    (
        test_fp["model_f1"],
        test_drcif["model_f1"], 
        test_stc["model_f1"],
        test_tde["model_f1"],
        test_mr["model_f1"],
        test_mr_baseline["model_f1"],
        test_itc["model_f1"] 
    )
)
results["model"] = ["FreshPRINCE", "DrCIF", "STC", "TDE", "MR", "MR-B", "ITC"]
results = results.set_index("model")
results = results.rename(columns = {0: "machine", 1 : "time", 2: "operation"})
results["avg"] = results.mean(axis = 1)

In [27]:
results_recall = pd.DataFrame(
    (
        test_fp["model_recall"],
        test_drcif["model_recall"], 
        test_stc["model_recall"],
        test_tde["model_recall"],
        test_mr["model_recall"],
        test_mr_baseline["model_recall"],
        test_itc["model_recall"]
    )
)
results_recall["model"] = ["FreshPRINCE", "DrCIF", "STC", "TDE", "MR", "MR-B", "ITC"]
results_recall = results_recall.set_index("model")
results_recall = results_recall.rename(columns = {0: "machine", 1 : "time", 2: "operation"})
results_recall["avg"] = results_recall.mean(axis = 1)

In [31]:
results

,machine,time,operation,avg
model,,,,
FreshPRINCE,0.970874,0.981481,0.971429,0.974595
DrCIF,0.980392,0.971963,0.951456,0.967937
STC,0.865979,0.915888,0.927273,0.903047
TDE,0.970874,0.971963,0.872727,0.938521
MR,0.074074,0.000000,0.000000,0.024691
MR-B,0.970874,0.981481,0.981132,0.977829
ITC,0.980392,0.971963,0.962264,0.971540


In [29]:
results_recall

,machine,time,operation,avg
model,,,,
FreshPRINCE,0.961538,0.963636,0.944444,0.956540
DrCIF,0.961538,0.945455,0.907407,0.938133
STC,0.807692,0.890909,0.944444,0.881015
TDE,0.961538,0.945455,0.888889,0.931961
MR,0.038462,0.000000,0.000000,0.012821
MR-B,0.961538,0.963636,0.962963,0.962713
ITC,0.961538,0.945455,0.944444,0.950479


In [104]:
MODELS = [test_fp, test_drcif, test_stc, test_tde, test_mr, test_mr_baseline, test_itc]
modelnames = ["fp", "drcif", "stc", "tde", "mr", "mr_b", "itc"]
for model, name in zip(MODELS, modelnames):
    filename = f"test_results/{name}_test.pkl"
    with open(filename, "wb") as f: 
        pickle.dump(model, f)